In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import datetime
import numpy as np
import os
import pandas as pd
import random

from keras import losses, optimizers
from keras.layers import Activation, Dense

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from statistics import mean

from tensorflow import keras

from tensorflow.keras import backend as K
from tensorflow.keras import metrics, losses
from tensorflow.keras import layers
from tensorflow.keras import optimizers

from tensorflow.keras.activations import elu, exponential, hard_sigmoid, linear, \
    relu, sigmoid, softmax, tanh
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy, \
    logcosh, mean_squared_error, poisson, mean_absolute_error
from tensorflow.keras.optimizers import Adadelta, Adam, Nadam, RMSprop, SGD

Using TensorFlow backend.


In [2]:
os.chdir(r'C:\Users\mmandziej001\Desktop\FCU\SCRIPTS\predictive_qc_db')
random.seed(100)

In [3]:
best_model = load_model(r'models\results\nn\cat4\seconditeration\nn_results_boruta_2020-05-19.h5')

train_data = pd.read_csv(r'models/data/cat4/dataset_dummy_grouped_time_train_cat4_inc.csv')
test_data = pd.read_csv(r'models/data/cat4/dataset_dummy_grouped_time_test_cat4_inc.csv')

boruta_importance = pd.read_excel(r'data/ai_assistant_dumps/cat4/Boruta_variable_importance2.xlsx')
boruta_importance = boruta_importance.sort_values(by=['normHits', 'meanImp'],
                                                  ascending=True)

In [ ]:
print(len(train_data), sum(train_data['Label']), round(sum(train_data['Label'])/len(train_data), 3))
print(len(test_data), sum(test_data['Label']), round(sum(test_data['Label'])/len(test_data), 3))

"""
full_dataset = pd.concat([train_data, test_data])
train_data, test_data = train_test_split(full_dataset,
                                         test_size=0.2,
                                         shuffle=True,
                                         random_state=100)

print('New approach: ')
print(len(train_data), sum(train_data['Label']), round(sum(train_data['Label'])/len(train_data), 3))
print(len(test_data), sum(test_data['Label']), round(sum(test_data['Label'])/len(test_data), 3))

train_data.to_csv(r"models\results\nn\cat6\newapproach\train_data.csv")
test_data.to_csv(r"models\results\nn\cat6\newapproach\test_data.csv")
"""

In [5]:
print(len(train_data), sum(train_data['Label']), round(sum(train_data['Label'])/len(train_data), 3))
print(len(test_data), sum(test_data['Label']), round(sum(test_data['Label'])/len(test_data), 3))

1586 321 0.202
339 52 0.153


In [6]:
list(train_data)
#del train_data['Unnamed: 0']
#del test_data['Unnamed: 0']

['Unique',
 'analyst_cases_processed_30days',
 'analyst_cases_processed_90days',
 'analyst_cases_processed_all',
 'analyst_cases_average_score_all',
 'analyst_expierence_day',
 'analyst_score_deducted_1_all',
 'analyst_score_deducted_2_all',
 'analyst_score_deducted_4_all',
 'analyst_score_deducted_5_all',
 'analyst_score_deducted_6_all',
 'analyst_score_deducted_7_all',
 'analyst_score_deducted_8_all',
 'analyst_score_deducted_9_all',
 'analyst_score_deducted_10_all',
 'analyst_score_deducted_11_all',
 'analyst_score_deducted_12_all',
 'analyst_score_deducted_13_all',
 'analyst_score_deducted_14_all',
 'analyst_score_deducted_15_all',
 'analyst_score_deducted_17_all',
 'analyst_score_deducted_18_all',
 'analyst_error_binary_1_all',
 'analyst_error_binary_2_all',
 'analyst_error_binary_4_all',
 'analyst_error_binary_5_all',
 'analyst_error_binary_6_all',
 'analyst_error_binary_7_all',
 'analyst_error_binary_8_all',
 'analyst_error_binary_9_all',
 'analyst_error_binary_10_all',
 'analys

In [7]:
train_id = train_data.pop('Unique')
test_id = test_data.pop('Unique')

train_labels = train_data.pop('Label')
test_labels = test_data.pop('Label')

train_datestamp = train_data.pop('datestamp')
test_datestamp = test_data.pop('datestamp')

qc_columns = [col for col in train_data.columns if 'qc_' in col]
used_columns = [col for col in train_data.columns if col not in qc_columns]
boruta_features = list(
    boruta_importance[boruta_importance['decision'] == 'Confirmed'].V7)

In [8]:
test_data = test_data[boruta_features] # used_columns, boruta_features, models_cols
train_data = train_data[boruta_features] # used_columns, boruta_features, models_cols

test_predictions = best_model.predict(test_data)
train_predictions = best_model.predict(train_data)

test_predictions_df = pd.DataFrame(test_predictions)
train_predictions_df = pd.DataFrame(train_predictions)

# test results
test_results = pd.concat([test_id, test_datestamp, test_predictions_df, test_labels], axis=1)
test_results = test_results.rename(columns={0: "Score"})
test_results = test_results.sort_values(by='Score', axis=0, ascending=False)

SumTest = pd.DataFrame(np.cumsum(test_results['Label']))
SumTest = SumTest.rename(columns={'Label': 'Sum'})
test_results = pd.concat([test_results, SumTest], axis=1)
test_results.reset_index(inplace=True, drop=True)
X_coordinates = pd.DataFrame(np.arange(1,(test_results.shape[0]+1))/(test_results.shape[0]))
X_coordinates = X_coordinates.rename(columns={0: 'X_coordinates'})
y_coordinates = pd.DataFrame(test_results['Sum']/test_results['Sum'].max())
y_coordinates = y_coordinates.rename(columns={'Sum': 'y_coordinates'})
test_results = pd.concat([test_results, X_coordinates,y_coordinates], axis=1)

# train results
train_results = pd.concat([train_id, train_datestamp, train_predictions_df, train_labels], axis=1)
train_results = train_results.rename(columns={0:"Score"})
train_results = train_results.sort_values(by='Score', axis=0, ascending=False)

SumTrain = pd.DataFrame(np.cumsum(train_results['Label']))
SumTrain = SumTrain.rename(columns={'Label':'Sum'})
train_results = pd.concat([train_results, SumTrain], axis=1)

train_results.reset_index(inplace=True , drop=True)
X_coordinates = pd.DataFrame(np.arange(1,(train_results.shape[0]+1))/(train_results.shape[0]))
X_coordinates = X_coordinates.rename(columns={0: 'X_coordinates'})
y_coordinates = pd.DataFrame(train_results['Sum']/train_results['Sum'].max())
y_coordinates = y_coordinates.rename(columns={'Sum': 'y_coordinates'})
train_results = pd.concat([train_results, X_coordinates,y_coordinates], axis=1)

auc_test = roc_auc_score(test_labels, test_predictions_df)
auc_train = roc_auc_score(train_labels, train_predictions_df)
print(auc_train, auc_test)

0.6933766761479073 0.7003819351380328


In [9]:
test_results.to_csv(r'models/results/nn/cat4/nn_test_results.csv')
train_results.to_csv(r'models/results/nn/cat4/nn_train_results.csv')